Importamos las librerías que necesitamos, incluimos el fichero de configuración de la conexión con Redmine

In [ ]:
import sys
from collections import OrderedDict
from pyexcel_ods import get_data
from config_rm import *
from redminelib import Redmine

Ahora procedemos a cargar la hoja de datos en una estructura data, sólo la pestaña correspondiente

In [ ]:
file_path = "input_wbs.ods"
data = get_data(file_path)['Export2Redmine']
print("file loaded")

Configuramos la carga, definiendo prefijos o números de fila o columna donde se encuentran los datos

In [ ]:
cfg_prefix = "w"
cfg_name_prefix = "WP"
cfg_tgt_prefix = "T"
cfg_first_valid_row = 3
cfg_import_column = 0
cfg_level_column = 4
cfg_ident_column = 1
cfg_name_column = 17
cfg_parent_column = 14
cfg_description_column = 18
cfg_assignee_column = 19
cfg_prefix_column = 20
cfg_input_column = 28

cfg_use_prjprefix_as_ident = True

Definimos cómo una fila tipo Target (Entregable "output") se carga en la estructura de datos "projects".
El entregable se definirá dentro del proyecto que llega en la columna "parent".

In [ ]:
def processTgt(row, projects):
    level = int(row[cfg_level_column])
    thisTgt = {}
    print("tgt:",row[cfg_ident_column])
    thisTgt['level'] = level
    # La descripción será la concatenación entre el nombre y la columna descripción (si existe), con unos retornos de carro por en medio
    thisTgt['description'] = row[cfg_name_column]
    if (len(row) > cfg_description_column):
        if (len(row[cfg_description_column])>0):        
            thisTgt['description'] += "\n\n" + row[cfg_description_column]
    
    if (len(row[cfg_parent_column])>0):
        # Si tiene proyecto padre definido, lo enlazamos, y generamos el identificador del target a partir del identificador del padre, según un número de orden que lo otorga la fila
        thisTgt['parent'] = row[cfg_parent_column]
        idx = projects[row[cfg_parent_column]]['tgtnum'] + 1
        thisTgt['ident'] = projects[row[cfg_parent_column]]['prefix'] + "."+cfg_tgt_prefix+"{:02d}".format(idx)
        projects[row[cfg_parent_column]]['tgtnum'] = idx
        projects[row[cfg_parent_column]]['targets'][row[cfg_ident_column]] = thisTgt

    else:
        # Si no tiene padre definido, se le asigna el proyecto raíz como padre
        thisTgt['parent'] = None
        idx = projects['root']['tgtnum'] + 1
        thisTgt['ident'] = cfg_tgt_prefix + "{:02d}".format(idx)
        projects['root']['tgtnum'] = idx
        projects['root']['targets'][row[cfg_ident_column]] = thisTgt
    
    # Devolvemos el nodo recién creado
    return thisTgt    

Definimos cómo una fila tipo Proyecto se carga en la estructura de datos "projects". El entregable se definirá dentro del proyecto que llega en la columna "parent".

In [ ]:
def processPrj(row, projects):
    level = int(row[cfg_level_column])
    thisPrj = {}
    print("prj:",row[1])
    thisPrj['level'] = level
    thisPrj['description'] = row[cfg_name_column]
    thisPrj['childnum'] = 0
    thisPrj['tgtnum'] = 0
    thisPrj['targets'] = {}
    # La descripción será la concatenación entre el nombre y la columna descripción (si existe), con unos retornos de carro por en medio
    if (len(row) > cfg_description_column):
        if (len(row[cfg_description_column])>0):
            thisPrj['description'] += "\n\n" + row[cfg_description_column]
    
    if (len(row[cfg_parent_column])>0):
        # Si tiene proyecto padre definido, lo enlazamos, y generamos el identificador del target a partir del identificador del padre, según un número de orden que lo otorga la fila
        thisPrj['parent'] = row[cfg_parent_column]
        idx = projects[row[cfg_parent_column]]['childnum'] + 1
        projects[row[cfg_parent_column]]['childnum'] = idx
        thisPrj['name'] = projects[row[cfg_parent_column]]['name'] + ".{:02d}".format(idx)
        thisPrj['ident'] = projects[row[cfg_parent_column]]['ident'] + "-{:02d}".format(idx)

    else:
        # Si no tiene padre definido, se le asigna el proyecto raíz como padre
        thisPrj['parent'] = None
        idx = projects['root']['childnum'] + 1
        projects['root']['childnum'] = idx
        thisPrj['name'] = cfg_name_prefix + "{:02d}".format(idx)
        thisPrj['ident'] = cfg_prefix + "{:02d}".format(idx)
    
    # Si tiene un responsable, lo asignamos al proyecto
    if (len(row) > cfg_assignee_column):
        if (len(row[cfg_assignee_column])>0):
            thisPrj['assignee'] = row[cfg_assignee_column]

    # Si existe un prefijo definida, lo cargamos en la estructura
    if (len(row) > cfg_prefix_column):
        if (len(row[cfg_prefix_column])>0):
            thisPrj['prefix'] = row[cfg_prefix_column]
            # Si tenemos habilitada la configuración de usar el prefijo definido como identificador del proyecto
            # sobreescribimos el identificador de proyecto a partir del prefijo definido
            if (cfg_use_prjprefix_as_ident):
                thisPrj['ident'] = cfg_prefix + thisPrj['prefix']
        
    # Devolvemos el nodo recién creado
    return thisPrj

Definimos el proceso de carga de cada fila, donde se discrimina si la fila corresponde a un proyecto o un target, y redirigimos a los tratamientos correspondientes arriba definidos

In [ ]:
def process_row(row, projects):
    level = int(row[cfg_level_column])
    if (level > 0):
        lastPrj = processPrj(row,projects)
        projects[row[cfg_ident_column]] = lastPrj
        print("-->",lastPrj['ident'])
    else:
        lastTgt = processTgt(row, projects)
        targets[row[cfg_ident_column]] = lastTgt
        print("-->",lastTgt['ident'])


Comenzamos la carga de la hoja de datos, consumiendo las filas de la estructura de datos, y llamando al procesado de la fila definido más arriba.

In [ ]:
# Creamos la estructura de proyectos, y creamos un proyecto raíz para contener los proyectos o targets que tienen padre
i = 0
projects = {}
projects['root'] = {}
projects['root']['childnum'] = 0
projects['root']['tgtnum'] = 0
projects['root']['targets'] = {}
targets = {}

# Comenzamos la carga
for row in data:
    # Let's skip the first row
    if (i >= cfg_first_valid_row):
        if (len(row) > cfg_import_column):
            if (row[cfg_import_column]>0):
                #print("processing row",i)
                process_row(row,projects)

    i += 1


Añadimos una función que hace "pretty printing" de diccionarios, para poder explorar los contenidos de los datos en las trazas

In [ ]:
def pretty(d, indent=0):
   for key, value in d.items():
      print('\t' * indent + str(key))
      if isinstance(value, dict):
         pretty(value, indent+1)
      else:
         print('\t' * (indent+1) + str(value))
            


Añadimos algunas trazas para analizar la estructura de datos cargada

In [ ]:
# pretty(projects)
# pretty(targets)

In [ ]:
# print(targets.keys())
# print(projects.keys())

Nos conectamos al Redmine y listamos sus proyectos.
Seleccionaremos el proyecto raíz a partir del que se ha configurado en el fichero de configuración de la conexión con Redmine

In [ ]:
redmine = Redmine(rm_server_url,key=rm_key_txt)
rmprojects = redmine.project.all()


# Durante la carga, vamos a mantener un diccionario con los proyectos de Redmine usando el identificador como llave
# este diccionario me permitirá saber de antemano qué proyectos están ya en el Redmine
rmprojectsbyident = {}
print("Proyectos:")
for p in rmprojects:
    print ("\t",p.identifier," \t| ",p.name)
    rmprojectsbyident[p.identifier] = p

root_rmproject = redmine.project.get(rm_project_id_str)
print ("Obtenemos proyecto raíz: ",root_rmproject.identifier," | ",root_rmproject.name)

Empezamos a crear los proyectos que no existan ya en el Redmine, o a modificar los existentes, según la información que hemos cargado de la hoja de datos.
También creamos los enlaces entre proyectos padres e hijos.
También crearemos los targets que no existan, o modificaremos los preexistentes.
ATENCIÓN: En ningún caso borramos proyectos o targets preexistentes.  Es importante vigilar que no queden restos de ejecuciones anteriores, ya que el programa no lo hace para evitar destrucción de datos en caso de error.
Mientras se está validando la carga, y la hoja de datos esté abierta a cambios, pueden quedar restos. Se recomienda, una vez finalizado el proceso, borrar todos los proyectos generados (borrando el proyecto raíz), y hacer una última ejecución limpia en el servidor de pruebas, y analizarla, antes de proceder a ejecutar la carga definitiva en el servidor de producción.

In [ ]:
# Recorremos los proyectos
for k in projects.keys():
    # Vamos a evitar el proyecto raíz
    if (k != 'root'):
        # Tomamos la información del proyecto en curso
        thisproject = projects[k]
        prident = thisproject['ident']
        print("Cargaremos el proyecto:",k,prident)
        rmprj = None
        if (prident in rmprojectsbyident.keys()):
            # Está en la lista de proyectos preexistentes, 
            # cargo sus datos del Redmine y los asocio a la estructura de datos
            rmprj = redmine.project.get(prident)
            if (rmprj is not None):
                projects[k]['rmprj'] = rmprj
        
        # Miro si el proyecto tiene padre o no
        if (thisproject['parent'] is None):
            # En caso de que no lo tenga, le asigno como padre el proyecto raíz
            rmprojectid = root_rmproject.id
            
        else:
            # En caso de que se haya definido un proyecto padre, lo asigno buscándolo en la estructura
            # por la forma de la hoja de datos, los proyectos padre siempre se cargan antes que los hijos,
            # por lo que tenemos garantía de éxito en la búsqueda.
            rmprojectid = projects[thisproject['parent']]['rmid']
        
        print("ID del proyecto padre: ",rmprojectid,thisproject['ident'])
        
        # Nos toca crear o modificar el proyecto de Redmine
        if rmprj is None:
            # El proyecto de Redmine no estaba en la lista de proyectos preexistentes
            # Esto significa que debe ser creado
            rmprj = redmine.project.create(
                    name=thisproject['name'],
                    identifier=thisproject['ident'],
                    description=thisproject['description'][:250],
                    is_public=True,
                    parent_id=rmprojectid,
                    inherit_members=True
            )
            
        else:
            # El proyecto ya existía en la lista de proyectos.
            # Debemos actualizarlo
            redmine.project.update(
                    rmprj.id,
                    name=thisproject['name'],
                    identifier=thisproject['ident'],
                    description=thisproject['description'][:250],
                    is_public=True,
                    parent_id=rmprojectid,
                    inherit_members=True
            )
        # Enlazamos el proyecto de Redmine con la estructura de datos del proyecto
        # Para que más tarde podamos llegar a él sin necesidad de búsquedas
        thisproject['rmid'] = rmprj.id
        thisproject['rmprj'] = rmprj

        # Ahora vamos a crear o modificar los "targets" del proyecto recién tratado
        for kt in thisproject['targets']:
            thistgt = targets[kt]
            tgtident = thistgt['ident']
            print("Vamos a procesar el target",kt,tgtident)
            tgtname = thistgt['ident'] + " " + thistgt['description'][:25]
        
            # Buscamos entre los targets del proyecto a ver si ya existe el target
            # que debemos procesar
            rmtgt = None
            for v in rmprj.versions:
                if v.name == tgtname:
                    # El target ya existe, no habrá que crearlo
                    rmtgt = v

            # Ahora procederemos a crear o modificar el target
            if rmtgt is None:
                print("Creamos el target")
                rmtgt = redmine.version.create(
                     project_id=rmprj.id,
                     name=tgtname,
                     description=thistgt['description'][:250]
                )            

            else:
                print("Modificamos el target")
                redmine.version.update(
                    rmtgt.id,
                    name=tgtname,
                    description=thistgt['description'][:250]
                )

            # En la estructura de datos del target recién creado enlazamos el objeto Redmine
            # para poder acceder después a él sin necesidad de buscarlo en el Redmine
            thistgt['rmtgt'] = rmtgt


A la hora de asignar responsables a las tareas o a los proyectos, necesitamos saber que los usuarios ya existen en el Redmine.
Para ello construiremos un diccionario de usuarios preexistentes, usando su login como llave.

In [ ]:
rmusers = redmine.user.all()
users = {}
for u in rmusers:
    #print(u.login)
    users[u.login] = u
    
print(users)

A la hora de asignar responsables a los proyectos, necesitamos saber que el rol de manager ya existe en el Redmine.
Para ello obtendremos del Redmine el rol adecuado, para luego poderlo asignar al responsable.

In [ ]:
roles = redmine.role.all()
for r in roles:
    print(r.name)
    if (r.name == 'RmMngr'):
        mngrole = r

A la hora de crear las tareas, necesitamos tener el tracker adecuado para ellas, de entre los existentes en el Redmine.
El tracker adecuado para cada tipo de tarea lo almacenamos para usarlo después.
NOTA: De momento usamos un tracker genérico, más adelante podría decidirse usar variables de configuración que permitan seleccionar un tracker diferente para cada tipo de tarea.

In [ ]:
trackers = redmine.tracker.all()
for tr in trackers:
    print(tr.name)
    if (tr.name == 'wrk'):
        ko_tracker = tr
        dv_tracker = tr
        cl_tracker = tr
        

A continuación, para cada proyecto, vamos a buscar su responsable del diccionario de usuarios obtenido del Redmine, y vamos a asignarle el rol de manager del proyecto

In [ ]:
for k in projects.keys():
    # Evitamos reconfigurar el proyecto raíz
    if (k!='root'):
        # Obtenemos el proyecto de Redmine a partir de la estructura
        p = projects[k]
        thispr = p['rmprj']
        print("Asignamos usuario",p['assignee'],"a proyecto:",k,p['ident'])
        if p['assignee'] not in users.keys():
            print("ERROR no existe el usuario",p['assignee'],"en el Redmine!!!")
            
        else:
            u = users[p['assignee']]
            # Vamos a listar las membresías del equipo y ver si el usuario ya está en ellos con el rol adecuado
            created = False
            members = thispr.memberships
            for m in members:
                if (m.user.id == u.id):
                    print("El usuario ya forma parte del proyecto")
                    created = True
                    done = False
                    thisroles = []
                    for r in m.roles:
                        thisroles += [r.id]
                        if mngrole.id == r.id:
                            print("El usuario ya tiene el rol correcto asignado, nada que hacer")
                            done = True
                    
                    if not done:
                        print("El usuario está en el proyecto, pero no con el rol adecuado, modificamos su membresía para añadirlo")
                        thisroles += [mngrole.id]
                        m2 = redmine.project_membership.get(m.id)
                        redmine.project_membership.update(m.id, role_ids=[1, 2])
            
            if not created:
                print("El usuario no formaba parte del proyecto, lo añadimos con el rol adecuado")
                redmine.project_membership.create(project_id=p['rmid'], user_id=users[p['assignee']].id, role_ids=[mngrole.id])


A continuación, para cada target vamos a crear las tres tareas asociadas (kick-off, desarrollo y cierre), asignarles como asignado el responsable del proyecto, y las enlazaremos entre sí con relaciones de precedencia 

In [ ]:
for k in projects.keys():
    # Evito tratar el proyecto raíz
    if (k!='root'):
        p = projects[k]['rmprj']
        print("Crearemos las tareas ko, dev, cl para el proyecto ",p.identifier)
        # En la estructura de datos del proyecto tendremos un diccionario con las tareas creadas, para luego poder llegar a ellas sin realizar búsquedas en Redmine
        issues = {}
        # El usuario asociado a las tareas es el responsable del proyecto
        # NOTA: podría considerarse usar la columna de responsable del target, pero entonces habría que asegurar que el responsable del target
        # está en el equipo de proyecto con un rol adecuado (developer?)
        u = users[projects[k]['assignee']]
        
        # Antes de comenzar a crear tareas, vamos a añadir las preexistentes al diccionario de tareas del proyecto en la estructura de datos
        for i in p.issues:
            issues[i.subject] = i
            print("--> Tarea preexistente",i.subject)
        
        # Ahora comenzamos a recorrer los targets del proyecto, para los cuales queremos crear/modificar la terna de tareas
        for thistgt in projects[k]['targets']:
            t = targets[thistgt]['rmtgt']
            print("Target -->",k,t,p.identifier)
            
            # Creamos o modificamos la tarea de kickoff
            ko_subject = "Kick-off " + t.name
            ko_descr = "Kick-Off meeting for deliverable named '"+t.name+"'"
            if ko_subject in issues.keys():
                print("La tarea a crear ya existe entre las tareas del proyecto, la actualizo")
                ko_i = issues[ko_subject]
                redmine.issue.update(
                    ko_i.id,
                    tracker_id=ko_tracker.id,
                    description=ko_descr,
                    fixed_version_id=t.id,
                    assigned_to_id=u.id          
                )
            else:
                print("La tarea a crear no existe entre las tareas del proyecto, la creo")
                ko_i = redmine.issue.create(
                    project_id=p.id,
                    subject=ko_subject,
                    tracker_id=ko_tracker.id,
                    description=ko_descr,
                    fixed_version_id=t.id,
                    assigned_to_id=u.id
                )
                # Añado la tarea al diccionario, para que ya conste
                issues[ko_subject] = ko_i
            
            # Registro la tarea de kick-off en la estructura de datos del target, para poder llegar a ella sin realizar búsquedas
            targets[thistgt]['ko_tsk'] = ko_i
            
            # Creamos o modificamos la tarea de desarrollo
            ko_dv_rel_done = False
            dv_subject = "Development " + t.name
            dv_descr = "Development for deliverable named '"+t.name+"'"      
            if dv_subject in issues.keys():
                print("La tarea a crear ya existe entre las tareas del proyecto, la actualizo")
                dv_i = issues[dv_subject]
                redmine.issue.update(
                    dv_i.id,
                    tracker_id=dv_tracker.id,
                    description=dv_descr,
                    fixed_version_id=t.id,
                    assigned_to_id=u.id   
                )
                # Vamos a ver si la dependencia con la tarea de kick-off ya existe, para 
                # evitar crearla más tarde en tal caso
                # NOTA: No estamos comprobando que exista con el mismo tipo de relación
                for r in dv_i.relations:
                    if r.issue_id == ko_i.id:
                        ko_dv_rel_done = True
            else:
                print("La tarea a crear no existe entre las tareas del proyecto, la creo")                
                dv_i = redmine.issue.create(
                    project_id=p.id,
                    subject=dv_subject,
                    tracker_id=dv_tracker.id,
                    description=dv_descr,
                    fixed_version_id=t.id,
                    assigned_to_id=u.id
                )
                # Añado la tarea al diccionario, para que ya conste
                issues[dv_subject] = dv_i
            
            # Registro la tarea de desarrollo en la estructura de datos del target, para poder llegar a ella sin realizar búsquedas
            targets[thistgt]['dv_tsk'] = dv_i
            
            # Si no está ya creada, deberemos añadir la relación de dependencia entre esta tarea y la anterior
            if not ko_dv_rel_done:
                # Create the relation
                ko_dv_rel = redmine.issue_relation.create(
                    issue_id=ko_i.id,
                    issue_to_id=dv_i.id,
                    relation_type='precedes',
                    delay=0
                )

            # Creamos o modificamos la tarea de cierre
            dv_cl_rel_done = False            
            cl_subject = "Closure " + t.name
            cl_descr = "Closure for deliverable named '"+t.name+"'"                    
            if cl_subject in issues.keys():
                print("La tarea a crear ya existe entre las tareas del proyecto, la actualizo")
                cl_i = issues[cl_subject]
                redmine.issue.update(
                    cl_i.id,
                    tracker_id=cl_tracker.id,
                    description=cl_descr,
                    fixed_version_id=t.id,
                    assigned_to_id=u.id          
                )
                # Vamos a ver si la dependencia con la tarea de kick-off ya existe, para 
                # evitar crearla más tarde en tal caso
                # NOTA: No estamos comprobando que exista con el mismo tipo de relación                
                for r in cl_i.relations:
                    print(dict(r))
                    if r.issue_id == dv_i.id:
                        dv_cl_rel_done = True                
            else:
                print("La tarea a crear no existe entre las tareas del proyecto, la creo") 
                cl_i = redmine.issue.create(
                    project_id=p.id,
                    subject=cl_subject,
                    tracker_id=cl_tracker.id,
                    description=cl_descr,
                    fixed_version_id=t.id,
                    assigned_to_id=u.id
                )
                # Añado la tarea al diccionario, para que ya conste
                issues[cl_subject] = cl_i
                
            # Registro la tarea de desarrollo en la estructura de datos del target, para poder llegar a ella sin realizar búsquedas
            targets[thistgt]['cl_tsk'] = cl_i
            
            # Si no está ya creada, deberemos añadir la relación de dependencia entre esta tarea y la anterior
            if not dv_cl_rel_done:
                # Create the relation
                dv_cl_rel = redmine.issue_relation.create(
                    issue_id=dv_i.id,
                    issue_to_id=cl_i.id,
                    relation_type='precedes',
                    delay=0
                )

After loading all the projects and targets, and creating all the issues, we must re-explore the input file to extract the relationships between the targets.
Definimos la función que procesará cada fila de la hoja de datos.

In [ ]:
# Esta función trata una fila buscando inputs.  También va manteniendo el puntero lastPrj para que, cuando se encuentre un "input" sepamos a qué 
# proyecto pertenece
def process_row_inputs(row, lastPrj, projects):
    done = False
    # La columna "import" puede ser False para un "input", pero para una fila que define un proyecto siempre ha de ser "true"
    if (len(row) > cfg_import_column):
        print(row[cfg_import_column])
        if (row[cfg_import_column]>0):
            # La columna de importación es True, ¿es un proyecto o un input?
            level = int(row[cfg_level_column])
            if (level > 0):
                # El nivel es mayor que cero, así que es una fila que define un proyecto, lo guardaremos en el puntero lastPrj
                lastPrj = projects[row[cfg_ident_column]]
                print("prj -->",lastPrj['ident'])
                # Marcamos el proceso como finalizado, para que no se busque un input en esta fila
                done = True

    if not(done):
        # La fila no correspondía a un proyecto, por lo que podemos buscar si hay un "input" definido en ella
        if (len(row) > cfg_input_column):
            if (len(row[cfg_input_column])>0):
                # La fila actual define un "input", obtenemos la información necesaria para poder enlazar este input con los targets del proyecto actual que apunta lastPrj.
                print("El input ",row[cfg_input_column],"(",targets[row[cfg_input_column]]['ident'],targets[row[cfg_input_column]]['rmtgt'],") ha de bloquear los targets del proyecto ",lastPrj['ident'],lastPrj['rmid'])
                inputko = targets[row[cfg_input_column]]['ko_tsk']
                inputdv = targets[row[cfg_input_column]]['dv_tsk']
                inputcl = targets[row[cfg_input_column]]['cl_tsk']
                print("++ Estas son las tareas del input ko:",inputko.id,"dv:",inputdv.id,"cl:",inputcl.id)

                # Ahora recorreremos los targets de lastPrj, para obtener sus tareas, y así poder crear las dependencias R1 y R2
                for t in lastPrj['targets']:
                    print("-- Proceso el target \\",targets[t])
                    # Obtenemos las tareas del target "output"
                    outputko = targets[t]['ko_tsk']
                    outputdv = targets[t]['dv_tsk']
                    outputcl = targets[t]['cl_tsk']
                    print("** Las tareas de este output son ko:",outputko.id,"dv:",outputdv.id,"cl:",outputcl.id)
                    # We have to find or create the relationships R1 block-block from inputdv to outputdv
                    r1_rel_done = False
                    for r in outputdv.relations:
                        if r.issue_id == inputdv.id:
                            r1_rel_done = True
                            print("La relación R1 entre este input y este output ya existía")

                    if not r1_rel_done:
                        # Create the relation
                        rq_rel = redmine.issue_relation.create(
                            issue_id=inputdv.id,
                            issue_to_id=outputdv.id,
                            relation_type='blocks'
                        )
                        print("La relación R1 entre este input y este output acaba de crearse")


                    # We have to find or create the relationships R2 block block from inputcl to outputcl
                    r2_rel_done = False              
                    for r in outputcl.relations:
                        if r.issue_id == inputcl.id:
                            r2_rel_done = True
                            print("La relación R2 entre este input y este output ya existía")

                    if not r2_rel_done:
                        # Create the relation
                        rq_rel = redmine.issue_relation.create(
                            issue_id=inputcl.id,
                            issue_to_id=outputcl.id,
                            relation_type='blocks'
                        )                  
                        print("La relación R1 entre este input y este output acaba de crearse")
        
    return lastPrj

Lanzamos el proceso que toma la hoja de datos y lanza el proceso de tratamiento de los inputs definido con anterioridad

In [ ]:
i = 0
for row in data:
    # Let's skip the first row
    if (i >= cfg_first_valid_row):
        print("processing row",i)
        lastPrj = process_row_inputs(row,lastPrj,projects)
    
    i = i + 1 

Escribimos "done" para indicar que el proceso acabó con éxito.

In [ ]:
print("done")